In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [8]:
trg_fake=pd.read_csv(r"C:\Users\amany\Desktop\archive datasets\fake-news\train.csv")
print(trg_fake.shape)
trg_fake.head()

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
trg_fake.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
trg_fake.dropna(inplace=True)

In [11]:
X = trg_fake['title']
y = trg_fake['label']

In [13]:
X.isnull().sum(),y.isnull().sum()

(0, 0)

In [14]:
#vocublary size
voc_size=5000

In [20]:
messeges=X.copy()
messeges=pd.DataFrame(messeges,columns=['title'])
messeges['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messeges.reset_index(inplace=True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

F:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
F:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## DATA CLEANSING

In [24]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
new_data=[]
for i in range(0,len(messeges)):
    data=re.sub("[^a-zA-Z]"," ",messeges['title'][i])
    data=data.lower()
    data=data.split()
    data=[ps.stem(me) for me in data if me not in set(stopwords.words('english'))]
    data=' '.join(data)
    new_data.append(data)

In [25]:
new_data

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## ONE_HOT

In [26]:
one_hot_repr=[one_hot(me,voc_size) for me in new_data]
one_hot_repr

[[2694, 3468, 3459, 4045, 1397, 489, 795, 1102, 1918, 1963],
 [1682, 4168, 716, 2599, 639, 876, 3794],
 [4037, 13, 2242, 3977],
 [4513, 1051, 4748, 4133, 3815, 1283],
 [3220, 639, 1156, 2200, 2056, 545, 639, 4228, 3123, 183],
 [1471,
  3266,
  4714,
  2555,
  1796,
  3946,
  1594,
  106,
  686,
  4672,
  3698,
  465,
  1286,
  1877,
  3794],
 [3475, 2364, 26, 3814, 4380, 4863, 4701, 925, 1679, 2200, 3578],
 [1442, 4948, 2572, 1219, 3136, 185, 3946, 4813, 1679, 2200, 3578],
 [3993, 4717, 2662, 1867, 1890, 1791, 2037, 3389, 3946, 1416],
 [1035, 1342, 2721, 1507, 113, 4781, 2335, 4496],
 [225, 2453, 208, 67, 4355, 3356, 268, 453, 917, 3368, 458],
 [4133, 3406, 1397, 1791, 3946, 3136],
 [3921, 4358, 1244, 2252, 1520, 4888, 1546, 2163, 4979],
 [4699, 37, 629, 2329, 3286, 1903, 4415, 1679, 2200, 3578],
 [1541, 131, 1098, 658, 321, 1679, 2200, 3578],
 [4561, 4143, 3634, 4526, 1835, 2427, 84, 1260, 3221, 3604],
 [3013, 260, 4168],
 [3189, 2359, 2494, 4924, 3946, 298, 3603, 3794],
 [1524, 2414,

In [27]:
data=pad_sequences(one_hot_repr,padding='pre',maxlen=20)
data

array([[   0,    0,    0, ..., 1102, 1918, 1963],
       [   0,    0,    0, ...,  639,  876, 3794],
       [   0,    0,    0, ...,   13, 2242, 3977],
       ...,
       [   0,    0,    0, ..., 1679, 2200, 3578],
       [   0,    0,    0, ..., 1993,    1, 1933],
       [   0,    0,    0, ..., 1617,  832, 3535]])

## MODEL BUILDING

In [29]:
embedding_vector_features=40

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=20))
model.add(LSTM(100))   # 100 Neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [35]:
x=data.copy()
len(x),len(y)

(18285, 18285)

In [36]:
X=np.array(x)
y=np.array(y)

In [39]:
X.shape,y.shape

((18285, 20), (18285,))

## Data Splitting

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 16ms/step - loss: 0.4827 - accuracy: 0.7507 - val_loss: 0.1951 - val_accuracy: 0.9176
Epoch 2/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1398 - accuracy: 0.9468 - val_loss: 0.1935 - val_accuracy: 0.9233
Epoch 3/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0929 - accuracy: 0.9684 - val_loss: 0.2284 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0629 - accuracy: 0.9785 - val_loss: 0.2396 - val_accuracy: 0.9143
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0366 - accuracy: 0.9889 - val_loss: 0.2808 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0293 - accuracy: 0.9916 - val_loss: 0.3225 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0146 - accuracy: 0.9968 - val_loss: 0.3620 - val_accuracy: 0.9107

In [42]:
y_pred=model.predict_classes(X_test)

F:\anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [44]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9110190555095278
[[3110  309]
 [ 228 2388]]
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



## ADDING DROPOUT

In [48]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=20))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 17ms/step - loss: 0.4795 - accuracy: 0.7555 - val_loss: 0.1906 - val_accuracy: 0.9180
Epoch 2/10
192/192 [==============================] - 4s 18ms/step - loss: 0.1482 - accuracy: 0.9430 - val_loss: 0.1854 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0947 - accuracy: 0.9662 - val_loss: 0.2022 - val_accuracy: 0.9201
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0768 - accuracy: 0.9734 - val_loss: 0.2267 - val_accuracy: 0.9162
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0554 - accuracy: 0.9821 - val_loss: 0.2593 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0368 - accuracy: 0.9892 - val_loss: 0.3111 - val_accuracy: 0.9120
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0302 - accuracy: 0.9893 - val_loss: 0.3628 - val_accuracy: 0.9135

In [49]:
y_pred=model.predict_classes(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9181441590720796
[[3078  341]
 [ 153 2463]]
              precision    recall  f1-score   support

           0       0.95      0.90      0.93      3419
           1       0.88      0.94      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035

